## Структурна лінгвістика

#### 1. Побудуйте ланцюжок походження слів за зразком:

```
оженитися: (о + (женити + ся))
```

1. атиповий
2. мореплавання
3. оподаткувати
4. перевтілитися
5. схилившись
6. підсніжник

1. (а + (тип + ов + ий))
2. (((море) + плав) + анн + я)
3. ((о + (податк + ува)) + ти) - тут трохи складний випадок. є слово "подат+ок з похідними "податк+ов+ий". слова "податкувати" немає, воно вживаєтсья лише із префіксом о-.
4. (пере + (втіл + итися))
5. ((с + (хил + ив)) + шись)
6. (під + сніж + ник) - корінь "сні(г|ж)" - самостійне слово. префік під- і суфікс -ник додаються одночасно, оскільки ні слова "підсніг", ані "сніжник" не існує.

#### 2. Перевірте роботу [SnowballStem](http://snowballstem.org/) для спільнокореневих слів. Напишіть ваші спостереження.

1. truth, truthful, truthfulness, countertruth, untruthful, truthology
2. flaw, flaws, flawed, flawless, flawlessness, flawlessly, 
3. лес, лесной, лесник, лесничий, лесничество, пролесье
4. окно, окошко, подоконник, оконный, окнище


In [10]:
from nltk.stem.snowball import SnowballStemmer as stemmer

d = ', ' # shorthand for delimiter

w1 = 'truth, truthful, truthfulness, countertruth, untruthful, truthology'.split(d)
w2 = 'flaw, flaws, flawed, flawless, flawlessness, flawlessly'.split(d)
w3 = 'лес, лесной, лесник, лесничий, лесничество, пролесье'.split(d)
w4 = 'окно, окошко, подоконник, оконный, окнище'.split(d)

def stem_words(word_list, stemmer):
    for word in word_list:
        print(stemmer.stem(word), end=' ')
    print() # new line

stemmer_en = stemmer('english')
stem_words(w1, stemmer_en)
stem_words(w2, stemmer_en)

stemmer_ru = stemmer('russian')
stem_words(w3, stemmer_ru)
stem_words(w4, stemmer_ru)

w5 = 'беззащитный, беспристрастный, беспристрастность, разнимать, недописанный, перекличка'.split(d)
stem_words(w5, stemmer_ru)

truth truth truth countertruth untruth trutholog 
flaw flaw flaw flawless flawless flawless 
лес лесн лесник леснич лесничеств пролес 
окн окошк подоконник окон окнищ 
беззащитн беспристрастн беспристрастн разнима недописа перекличк 


##### 2 - Спостереження

З прикладів 1 і 2 бачимо, що стеммер впорався із суфіксами (-ful, -ness), але проігнорував деякі суфікси (-ology, -less) і геть усі префікси (un-, counter-, etc.). З побаченого можемо зробити припущення, що у стеммера є жорстко прописаний (hardcoded) перелік суфіксів, які він відсікає. 

І дійсно, при вивченні [вихідного коду](https://github.com/nltk/nltk/blob/15a181c5c043e5a9de709c63a58d2949d747b300/nltk/stem/snowball.py#L1266) англійського стеммера бачимо, що у нього прямо в коді є перелік суфіксів, які відсікаються на кожному кроці, а перелік префіксів відсутній.

На прикладах 3 і 4 спістерігаємо, що стеммер для російської мови добре відпрацьовує морфологічні закінчення, але некоректно відсікає деякі суфікси і (принаймні, у розглянутих прикладах) префікси. 

На додатковому прикладі 5 підтверджуємо, що стеммер Snowball для російської мови не бачить навіть найрозповсюдженіших префіксів. [Вихідний код](https://github.com/nltk/nltk/blob/15a181c5c043e5a9de709c63a58d2949d747b300/nltk/stem/snowball.py#L4520) підтверджує це припущення. 

На прикладі 4 також бачимо, що стеммер некоректно відпрацьовує випадки чергування голосних і приголосних.

* Також цікаво, що російський стеммер перед обробкою слова транслітерує його в кирилицю, а після обробки — назад :)

#### 3. Визначте частину мови виділеного слова і зв'язок до його батька (за зразком):

```
{I} like turtles.: pronoun, nsubj(like, I)  
I {like} turtles.: verb, root(ROOT, like)  
I like {turtles}.: noun, dobj(like, turtles)
```

1. We can {but} hope that everything will be fine.
2. It's sad {but} true.
3. Jack brings nothing {but} trouble.
4. {As} we were talking, I realised how to solve the issue.
5. This hot dog isn't {as} big as usual.
6. This hot dog isn't as big {as} usual.
7. This hot dog isn't as big {as} I expected.
8. I work {as} a teacher.
9. Let's do it this {way}!
10. This is {way} too much!
11. The prices are going {down}.
12. Someone pushed him and he fell {down} the stairs.
13. I’ve been feeling rather {down} lately.
14. It's not easy to {down} a cup of coffee in one gulp.
15. Bring a {down} jacket and a pair of gloves, and you'll be fine.



In [54]:
import spacy

sentences = {
    1: "We can {but} hope that everything will be fine.",
    2: "It's sad {but} true.",
    3: "Jack brings nothing {but} trouble.",
    4: "{As} we were talking, I realised how to solve the issue.",
    5: "This hot dog isn't {as} big as usual.",
    6: "This hot dog isn't as big {as} usual.",
    7: "This hot dog isn't as big {as} I expected.",
    8: "I work {as} a teacher.",
    9: "Let's do it this {way}!",
    10: "This is {way} too much!",
    11: "The prices are going {down}.",
    12: "Someone pushed him and he fell {down} the stairs.",
    13: "I’ve been feeling rather {down} lately.",
    14: "It's not easy to {down} a cup of coffee in one gulp.",
    15: "Bring a {down} jacket and a pair of gloves, and you'll be fine."
}

nlp = spacy.load('en_core_web_md')

for i in sentences.keys():
    doc = nlp(sentences[i])
    for token in doc:
        if token.text == "{":
            target_word = token.nbor(1)
            
            print(str(i) + ". "+ sentences[i]+":", end=' ')
            print(target_word.pos_, end=' ')
            print(target_word.dep_, end='(')
            print(target_word.head, end=', ')
            print(target_word.text, end=')\n')


1. We can {but} hope that everything will be fine.: CCONJ cc(can, but)
2. It's sad {but} true.: CCONJ cc(sad, but)
3. Jack brings nothing {but} trouble.: CCONJ cc(nothing, but)
4. {As} we were talking, I realised how to solve the issue.: ADP prep(talking, As)
5. This hot dog isn't {as} big as usual.: ADV prep(is, as)
6. This hot dog isn't as big {as} usual.: ADP prep(big, as)
7. This hot dog isn't as big {as} I expected.: ADP prep(is, as)
8. I work {as} a teacher.: ADP prep(work, as)
9. Let's do it this {way}!: NOUN npadvmod(do, way)
10. This is {way} too much!: NOUN attr(is, way)
11. The prices are going {down}.: PUNCT dobj(going, down}.)
12. Someone pushed him and he fell {down} the stairs.: ADP advmod(fell, down)
13. I’ve been feeling rather {down} lately.: ADV advmod(feeling, down)
14. It's not easy to {down} a cup of coffee in one gulp.: ADV xcomp('s, down)
15. Bring a {down} jacket and a pair of gloves, and you'll be fine.: NOUN nmod(jacket, down)


##### 3 - Спостереження №1

При вивченні видачі цього аналізу ми бачимо низку помилок — у р. 1, 3, 7, 8, 11, 14 — навіть на середній моделі. Можливо, круглі дужки певним чином впливають на розпізнавання? Спробуємо прибрати їх з аналізу і перевірити, чи зміниться точність розпізнавання моделі.

In [52]:
nlp = spacy.load('en_core_web_md')

for i in sentences.keys():
    doc = nlp(sentences[i])
    for token in doc:
        if token.text == "{":
            target_word = token.nbor(1).text
            # The `cont` flag will prevent processing  
            # multiple similar words in the same sentence
            cont = True
            break
            
    if target_word != '':
        debracketized_sentence = sentences[i].replace('{', '').replace('}', '')
        doc = nlp(debracketized_sentence)
        for token in doc:
            if token.text == target_word and cont:
                print(str(i) + ". "+ sentences[i]+":", end=' ')
                print(token.pos_, end=' ')
                print(token.dep_, end='(')
                print(token.head, end=', ')
                print(token.text, end=')\n')   
                cont = False
            
    else:
        raise "No word tagged for analysis."


1. We can {but} hope that everything will be fine.: CCONJ cc(can, but)
2. It's sad {but} true.: CCONJ cc(sad, but)
3. Jack brings nothing {except} trouble.: ADP prep(nothing, except)
4. {As} we were talking, I realised how to solve the issue.: ADP mark(talking, As)
5. This hot dog isn't {as} big as usual.: ADV advmod(big, as)
6. This hot dog isn't as big {as} usual.: ADV advmod(big, as)
7. This hot dog isn't as big {as} I expected.: ADV advmod(big, as)
8. I work {as} a teacher.: ADP prep(work, as)
9. Let's do it this {way}!: NOUN npadvmod(do, way)
10. This is {way} too much!: ADV advmod(much, way)
12. Someone pushed him and he fell {down} the stairs.: ADP prep(fell, down)
13. I’ve been feeling rather {down} lately.: ADV advmod(feeling, down)
14. It's not easy to {down} a cup of coffee in one gulp.: ADV xcomp('s, down)
15. Bring a {down} jacket and a pair of gloves, and you'll be fine.: PART amod(jacket, down)


##### 3 - Спостереження №2 і результати

Припущення, що дужки вплинули на розпізнавання частин мови та синтаксичних зв’язків, виявилося вірним. Однак, незважаючи на дещо кращі результати, у видачі залишилися помилки.

Після ручного аналізу приходимо до таких висновків:

1. : ADV advmod(can, but)
2. : CCONJ cc(sad, but)
3. : ADP prep(nothing, but)
4. : CCONJ cc(talking, As)
5. : ADV prep(is, as)
6. : ADP prep(big, as)
7. : ADP prep(big, as)
8. : ADP prep(work, as)
9. : NOUN npadvmod(do, way)
10. : NOUN attr(is, way)
11. : ADV dobj(going, down}.)
12. : ADP advmod(fell, down)
13. : ADV advmod(feeling, down)
14. : ADV xcomp('s, down)
15. : NOUN nmod(jacket, down)


#### 4. Визначте частину мови виділеного слова і зв'язок до його батька (за зразком):

```
{Я} люблю черепашок.: займенник, nsubj(люблю, Я)  
Я {люблю} черепашок.: дієслово, root(ROOT, люблю)  
Я люблю {черепашок}.: іменник, dobj(люблю, черепашок)  
```

1. Рада міністрів Європейського союзу затвердила угоду про спрощений порядок видачі {віз} для України.: іменник, dobj(видачі, віз)
2. Батько Себастьяна {віз} на санях їх театральний гурт до Львова.: дієслово, root(віз, ROOT)
3. А ще дивний елемент інтер’єру – {віз} із продукцією одного з херсонських виробників. noun, nsubj(віз, ROOT)
4. У цю мить {повз} Євгена пролетів останній вагон товарняка. прийменник, prep(Євгена, повз)
5. Кліпнув очима і побачив малого песика, який саме пробігав {повз} у бік села. прийменник, prep(пробігав, повз
6. Степанко перестав кричати, тільки ламкий стогін {повз} йому із грудей. дієслово, conj(повз, стогін). тут ’повз’ - граматичний присудок другого підрядного речення.
7. Часу не {гай} – декларацію подай! дієслово, root(гай, ROOT)
8. І коляд заспівали, і {гай} врятували. іменник, dobj(врятували, гай)
9. {Гай}, чи ви забулися, братове? вигук, intj(гай, гай)
10. Ось присіла на {край} ліжка. прийменник, pobj (ліжка, край)
11. Поставив ту кузню не {край} дороги, як було заведено, а на Красній горі, біля Прадуба. прийменник, pobj (дороги, край)
12. Розповідаючи про передній {край} лінґвістики, фон Лібіх, як завжди, мислив широко і глобально. прийменник, pobj(про, край)
13. Не {край} мені серце. дієслово, root(ROOT, край)
14. І {щойно} тоді додаємо до борщу смажену цибулю., прислівник, advmod(тоді, щойно)
15. Бо {щойно} я задрімав, віддавши тіло подушці з периною, як мене розбудив поштовх у бік. підрядний сполучник часу sconj(задрімав, щойно)

#### 5. Виберіть одне речення зі списку та побудуйте для нього дерево залежностей та дерево складників:

(Можна намалювати дерева в графічному редакторі або на папері і докріпити картинку до домашнього завдання. Назви складників, частин мови та залежностей можна не писати. Головне - це структура.)

1. Пригадую, уже згодом, коли я відбував свій термін у таборі № 36 у Кучино Пермської області, я отримав від Михасі листівку з жартівливим описом того, як Київ готується до святкування свого 1500-ліття.
2. 6C приземляється на плече, перекочуючись, пролітає метрів п’ятдесят і витягується на снігу за кілька кроків від забризканої палаючими уламками посадкової смуги.
3. Дівчина стояла там, де й була, і намагалася привести до ладу скуйовджене волосся, вкрай розлючена тим, що це побачили водії, які чекали на переїзді.


![1.5.1](1-5-1.png)
![1.5.2](1-5-2.png)

#### 6. Виберіть одне cлово зі списку та побудуйте лексико-семантичні зв'язки до трьох різних значень цього слова. Фактично, потрібно побудувати WordNet-подібні вузли. Значення слів можна перевірити у [СУМі](http://sum.in.ua/) та [Словниках України](http://lcorp.ulif.org.ua/dictua/).

Слова на вибір: вік, стіна, добро, серце, центр, куля, міст, ланцюг, бік, дух.

**Стіна**
_іменник;_

1. Вертикальна частина будови, яка служить для підтримання перекрить і для розділення приміщення на частини і т. ін. 

**Приклад:** Головою стіни не проб'єш

**Синоніми:** мур

**Гіпоніми:** перегородка, пристінок

2. чого і з означ. Прямовисна бічна поверхня чого-небудь. 

**Приклад:** Прямовисні стіни айсбергів підносяться над водою на 20—30 м, іноді і на 100 м

**Синоніми:** край

**Гіпоніми:** відкіс

3. чого і в означ. Висока кам'яна або цегляна огорожа; мур. 

**Приклад:** Перед вечором заманячіли в степу Лубни, вирнули з туману стіни твердині

**Синоніми:** вал, мур, огорожа, загорода, паркан

**Гіпоніми:** паркан, тин

4. чого і з означ., перен. Те, що відокремлює, роз'єднує кого-, що-небудь, перешкода у здійсненні чогось. 

**Приклад:** Ксеня почувала, що між нею та Олегом зростає невидима стіна відчуження...

**Синоніми:** перешкода, перепона, завада, загата

5. чого і з означ., перен. Щільний ряд чи суцільна маса чого-небудь, що утворює перепону, перешкоду, завісу і т. ін. 

**Приклад:** Усе лякало нас: трава, що тихо майоріла, і гаю дальнього стіна

**Синоніми:** завіса, заслона (рідк.)


6. Щільно зімкнений стрій бійців у кулачному, рукопашному бою або бійці. 

**Приклад:** Хоч і вгнулася спочатку стіна руських воїнів під натиском угорських сотень, але ніхто не побіг назад

**Синоніми:** шеренга, стрій, лава, ряд
